In [8]:
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
#Reading the csv file.
df=pd.read_csv("forestfires.csv")

#************************************************************************

#Converting column "area" into binary and adding to df.
list1=[0 for i in range(517)]
count=0
for index,rows in df.iterrows():
    if rows["area"]==0:
        list1[index]=0
    else:
        list1[index]=1
area_b=pd.DataFrame(list1)
df=pd.concat([df,area_b],axis=1)
df.rename({0:"area_b"},axis=1,inplace=True)
df.drop(["area"],axis=1,inplace=True)

#Converting columns "day" and "month" into binary and adding to df.
le_Gender=LabelEncoder() #Creating reference of LabelEncoder.
df["month_b"]=le_Gender.fit_transform(df["month"])
df["day_b"]=le_Gender.fit_transform(df["day"])
df.drop(["month","day"],axis=1,inplace=True)

#***************************************************************************

#Removing outleirs of column "Y".
for index,row in df.iterrows():
    if (row["Y"]>6) or (row["Y"]<3) :
        df=df.drop(index)
        
#Removing outleirs of column "FFMC".
for index,row in df.iterrows():
    if (row["FFMC"]>96) or (row["FFMC"]<88) :
        df=df.drop(index)
        
#Removing outleirs of column "DMC".
for index,row in df.iterrows():
    if (row["DMC"]>215) or (row["DMC"]<5) :
        df=df.drop(index)
        
#Removing outleirs of column "DC".
for index,row in df.iterrows():
    if (row["DC"]>800) or (row["DC"]<450) :
        df=df.drop(index)
        
#Removing outleirs of column "ISI".
for index,row in df.iterrows():
    if (row["ISI"]>15.5) or (row["ISI"]<0) :
        df=df.drop(index)
        
#Removing outleirs of column "temp".
for index,row in df.iterrows():
    if (row["temp"]>30) or (row["temp"]<12) :
        df=df.drop(index)
        
#After seeing so many outliers in column "rain", dropping "rain".
df.drop(["rain"],axis=1,inplace=True)

#Removal of Outliers completed.

#***************************************************************************

#Normailzation of the features.
for column in ["X","Y","FFMC","DMC","DC","ISI","temp","RH","wind","month_b","day_b"]:
    min_=float(df[column].min())
    max_=float(df[column].max())
    for index,row in df.iterrows():
        df[column]=df[column].replace(row[column],(row[column]-min_)/(max_-min_))
        
#After normalization, column "month_b" has more than 90% of zeroes.
#Hence dropping column "month_b".
df.drop(["month_b"],axis=1,inplace=True)

#***************************************************************************

#Rearranging "area_b" to the last of the DataFrame.
tempdf=pd.DataFrame(df["area_b"])
df.drop(["area_b"],axis=1,inplace=True)
df["area_b"]=tempdf

#Renaming column "X" and "Y"
df.rename({'X':'loc_x','Y':'loc_y'},axis=1,inplace=True)

#****************************************************************************

x=df.drop(["area_b"],axis=1) #Independent Variables
y=df["area_b"] #Dependent Variable

#Importing common modules for all models.
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

while(True):
    print("MENU\n1.LS for Logistic Regression\n2.DT for Decision Tree\n3.KN for K-Nearest Neighbor\n4.NB for Naive Bayes\n5.exit for Exit")
    var=input("Enter Choice:")
    if var=="LS":
        from sklearn.linear_model import LogisticRegression
        #k_=int(input("Enter the number of top features to be added: "))
        #k_=4
        for k_ in [3,4]:
            #Feature selection in "x" on the basis of "k_"
            bestfeatures=SelectKBest(score_func=chi2,k=k_)
            fit=bestfeatures.fit(x,y)
            dfscores=pd.DataFrame(fit.scores_)
            dfscor=dfscores[0].sort_values(ascending=False)
            features=dfscor.index.values
            X=x.iloc[ : ,features[0:k_]]
        
            #Spliting "X" into test and train set.
            x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=100)
            
            #Calling LogisticRegression constructor.
            logmodel=LogisticRegression()
            #Training the model.
            logmodel.fit(x_train,y_train)
            #Predicting the model.
            y_pred=logmodel.predict(x_test)
            
            print("For Top ",k_," Features, the analysis of Logistic Regression is as follow:")
            #Calculating Accuracy Score.
            acc=accuracy_score(y_test,y_pred)
            print("Accuracy Score is: ",acc)
            
            #Printing Confusion Matrix
            print("Confusion Matrix is:\n",confusion_matrix(y_test,y_pred))
            #Printing Classification Report
            print("Classification Report is:\n",classification_report(y_test,y_pred))
            print("*******************************************************************************")
        print("Hence the best fit model using Logistic Regression is achieved by using Top 4  features with Accuracy Score ",acc)
        
    elif var=="DT":
        from sklearn.tree import DecisionTreeClassifier
        #k_=int(input("Enter the number of top features to be added: "))
        #k_=4
        for k_ in [3,4]:
            #Feature selection in "x" on the basis of "k_"
            bestfeatures=SelectKBest(score_func=chi2,k=k_)
            fit=bestfeatures.fit(x,y)
            dfscores=pd.DataFrame(fit.scores_)
            dfscor=dfscores[0].sort_values(ascending=False)
            features=dfscor.index.values
            X=x.iloc[ : ,features[0:k_]]
        
            #Spliting "X" into test and train set.
            x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=100)
            
            #Calling DecisionTreeClassifier constructor.
            classifier_entropy=DecisionTreeClassifier(criterion="entropy",random_state=100,max_depth=5,min_samples_leaf=5)

            #Training the model.
            classifier_entropy.fit(x_train,y_train)
            #Predicting the model.
            y_pred=classifier_entropy.predict(x_test)
            
            print("For Top ",k_," Features, the analysis of Decision Tree is as follow:")
            #Calculating Accuracy Score.
            acc=accuracy_score(y_test,y_pred)
            print("Accuracy Score is: ",acc)
            
            #Printing Confusion Matrix
            print("Confusion Matrix is:\n",confusion_matrix(y_test,y_pred))
            #Printing Classification Report
            print("Classification Report is:\n",classification_report(y_test,y_pred))
            print("*******************************************************************************")
        print("Hence the best fit model using Decision Tree is achieved by using Top 4 features with Accuracy Score ",acc)
                
    elif var=="KN":
        from sklearn.neighbors import KNeighborsClassifier
        #k_=int(input("Enter the number of top features to be added: "))
        #k_=4
        for k_ in [3,4]:
            #Feature selection in "x" on the basis of "k_"
            bestfeatures=SelectKBest(score_func=chi2,k=k_)
            fit=bestfeatures.fit(x,y)
            dfscores=pd.DataFrame(fit.scores_)
            dfscor=dfscores[0].sort_values(ascending=False)
            features=dfscor.index.values
            X=x.iloc[ : ,features[0:k_]]
        
            #Spliting "X" into test and train set.
            x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=100)
            
            #Calling KNeighborsClassifier constructor.
            import math as m
            b=int(m.sqrt(len(x))) #Taking square root of total number of instances.
            classifier=KNeighborsClassifier(n_neighbors=b,p=2,metric='euclidean')
            
            #Training the model.
            classifier.fit(x_train,y_train)
            #Predicting the model.
            y_pred=classifier.predict(x_test)
            
            print("For Top ",k_," Features, the analysis of K-Nearest Neighbor is as follow:")
            #Calculating Accuracy Score.
            acc=accuracy_score(y_test,y_pred)
            print("Accuracy Score is: ",acc)
            
            #Printing Confusion Matrix
            print("Confusion Matrix is:\n",confusion_matrix(y_test,y_pred))
            #Printing Classification Report
            print("Classification Report is:\n",classification_report(y_test,y_pred))
            print("*******************************************************************************")
        print("Hence the best fit model using K-Nearest Neighbor is achieved by using Top 4 features with Accuracy Score ",acc)
    elif var=="NB":
        from sklearn.naive_bayes import GaussianNB
        #k_=int(input("Enter the number of top features to be added: "))
        #k_=4
        for k_ in [3,4]:
            #Feature selection in "x" on the basis of "k_"
            bestfeatures=SelectKBest(score_func=chi2,k=k_)
            fit=bestfeatures.fit(x,y)
            dfscores=pd.DataFrame(fit.scores_)
            dfscor=dfscores[0].sort_values(ascending=False)
            features=dfscor.index.values
            X=x.iloc[ : ,features[0:k_]]
        
            #Spliting "X" into test and train set.
            x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=100)
            
            #Calling DecisionTreeClassifier constructor.
            model=GaussianNB()
            
            #Training the model.
            model.fit(x_train,y_train)
            #Predicting the model.
            y_pred=model.predict(x_test)
            
            print("For Top ",k_," Features, the analysis of Naive Bayes is as follow:")
            #Calculating Accuracy Score.
            acc=accuracy_score(y_test,y_pred)
            print("Accuracy Score is: ",acc)
            if k_==3:
                final_acc=acc
            #Printing Confusion Matrix
            print("Confusion Matrix is:\n",confusion_matrix(y_test,y_pred))
            #Printing Classification Report
            print("Classification Report is:\n",classification_report(y_test,y_pred))
            print("*******************************************************************************")
        print("Hence the best fit model using Naive Bayes is achieved by using Top 3 features with Accuracy Score ",final_acc)
    elif var=="exit":
        sys.exit(0)
    else:
        print("Wrong Input!!!")


MENU
1.LS for Logistic Regression
2.DT for Decision Tree
3.KN for K-Nearest Neighbor
4.NB for Naive Bayes
5.exit for Exit
Enter Choice:quit
Wrong Input!!!
MENU
1.LS for Logistic Regression
2.DT for Decision Tree
3.KN for K-Nearest Neighbor
4.NB for Naive Bayes
5.exit for Exit
Enter Choice:exit


SystemExit: 0

C:\Users\Prabal\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


NameError: name 'X' is not defined